In [2]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import librosa
%matplotlib inline

##Load test_labels

In [3]:
path = '/content/drive/MyDrive/lables_test'
os.chdir(path)
os.getcwd()

'/content/drive/MyDrive/lables_test'

In [4]:
def eventroll_to_multihot_vector(eventroll):
    """
    Parameters
    ----------
    eventroll : np.array
        Eventroll matrix of shape=(11, 1000).
    
    Returns
    -------
    np.array
        A multihot vector of shape=(10,)
    """
    
    # findout active events:
    active_events = (eventroll.sum(axis=1) >= 0.5).astype('float')
    
    # remove silence class:
    return np.delete(active_events, 8)

In [5]:
os.chdir(path)
os.getcwd()

'/content/drive/MyDrive/lables_test'

In [6]:
yp = os.listdir()
len(yp)

2500

In [7]:
yp[0]

'eventroll_2348.npy'

In [8]:
yp.sort()

In [9]:
yp[0]

'eventroll_0.npy'

In [10]:
y_et = np.zeros((len(yp),11,1000))

for i in range(len(yp)):
  b = np.load(yp[i])
  y_et[i,:,:] = b[:,:]

In [11]:
y = np.zeros((len(yp),10))

for i in range(len(yp)):
  c = eventroll_to_multihot_vector(y_et[i])
  y[i,:] = c

In [12]:
y.shape

(2500, 10)

In [13]:
y[0].shape

(10,)

In [14]:
y[0]

array([0., 0., 0., 0., 1., 0., 0., 0., 1., 0.])

In [15]:
y_test = y

##Read X

In [16]:
os.getcwd()

'/content/drive/MyDrive/lables_test'

In [17]:
xtestpath = '/content/drive/MyDrive/test/X'

In [18]:
os.chdir(xtestpath)
os.getcwd()

'/content/drive/MyDrive/test/X'

In [19]:
xp = os.listdir()
len(xp)

2500

In [20]:
xp[0]

'melspec_20.npy'

In [21]:
xp.sort()

In [22]:
xp[0]

'melspec_0.npy'

In [23]:
x_test = np.zeros((len(xp),64,1000,1))

for i in range(len(xp)):
  a = np.load(xp[i])
  x_test[i,:,0:1000,0] = a[0,:,0:1000]

In [24]:
x_test.shape 

(2500, 64, 1000, 1)

In [25]:
xp[0]

'melspec_0.npy'

In [26]:
type(x_test)

numpy.ndarray

#Pred

In [27]:
m = tf.keras.models.load_model('/content/drive/MyDrive/cnn1')

In [28]:
y_pred = m.predict(x_test)

79/79 [==============================] - 32s 395ms/step


In [29]:
for i in range (len(y_pred)) :
  for j in range (10):
    if y_pred[i,j] >=0.5 :
      y_pred[i,j] = 1
    else :
      y_pred[i,j] = 0

In [30]:
y_pred[0]

array([0., 0., 0., 1., 0., 0., 0., 0., 1., 0.], dtype=float32)

In [31]:
y_pred = y_pred.astype(int)

In [32]:
y_pred

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0]])

In [33]:
y_pred[0]

array([0, 0, 0, 1, 0, 0, 0, 0, 1, 0])

#F1score

In [34]:
from sklearn.metrics import f1_score,recall_score,precision_score

In [36]:
print(f1_score(y_test,y_pred,average='samples'))

0.5810761904761905


#Recall

In [38]:
print(recall_score(y_test,y_pred,average='samples'))

0.4636704761904762


#Precision

In [40]:
print(precision_score(y_test,y_pred,average='samples'))

0.8566333333333332


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#Confusion Matrix

In [41]:
from sklearn.metrics import confusion_matrix
for i in range(10) :
  print(confusion_matrix(y_test[:,i], y_pred[:,i]))

[[2074   26]
 [ 393    7]]
[[2220   14]
 [ 263    3]]
[[2187   29]
 [ 281    3]]
[[1535  276]
 [ 566  123]]
[[2134   25]
 [ 336    5]]
[[2194   23]
 [ 280    3]]
[[2005  118]
 [ 354   23]]
[[2183   11]
 [ 305    1]]
[[   0  127]
 [   7 2366]]
[[2197   52]
 [ 249    2]]
